In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a forcasting problem, where the goal is to predict the number of cycles an engine will last before it fails.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [3]:
# third-party libraries
import pandas as pd
import numpy as np
import os

import time
from tqdm.notebook import tqdm

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [5]:
np.random.seed(42)

# Paths

In [6]:
# Make sure to execute this cell only once for one kernel session, before running any other cell below.
os.chdir("../") # set working directory to root of project
os.getcwd() # check current working directory

'/Users/frbroy/Library/Mobile Documents/com~apple~CloudDocs/KIT/SoSe2024/PSDA/damage-propagation-modeling'

In [7]:
# source code
from src.utils import load_data, load_config, train_val_split_by_group
from src.data_preprocessing import calculate_RUL
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [8]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [9]:
train_data, test_data, test_data_RUL = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)
train_data = calculate_RUL(train_data, time_column= "Cycle", group_column= "UnitNumber")

#train_data, val_data = train_val_split_by_group(train_data)

2024-05-17 00:14:34 [src.utils:62] [INFO] >>>> Loading data set 1...
2024-05-17 00:14:34 [src.utils:91] [INFO] >>>> Loaded raw data for dataset 1.
2024-05-17 00:14:34 [src.utils:92] [INFO] >>>> Train Data: (20631, 26)
2024-05-17 00:14:34 [src.utils:93] [INFO] >>>> Test Data: (13096, 26)
2024-05-17 00:14:34 [src.utils:94] [INFO] >>>> Test RUL Data: (100, 1)


# 📍 << Subtask NN: Try out NN Architecture >>

[TEMPLATE]

Findings:
* Interpretation of plots
* or other key take aways from previous code

In [10]:
train_data.head()

,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure 1,Sensor Measure 2,Sensor Measure 3,Sensor Measure 4,Sensor Measure 5,...,Sensor Measure 13,Sensor Measure 14,Sensor Measure 15,Sensor Measure 16,Sensor Measure 17,Sensor Measure 18,Sensor Measure 19,Sensor Measure 20,Sensor Measure 21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,192
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,191
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,190
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,189
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,188


In [57]:
## Scale Data Min Max
from sklearn.preprocessing import MinMaxScaler
def scale_data(df):

    scaler = MinMaxScaler()
    float_columns = df.select_dtypes(include=float).columns.tolist()
    scaled_data = scaler.fit_transform(df[float_columns])
    df[float_columns] = scaled_data

    return df

## create sliding window
def create_sliding_window(df, window_size = 30, drop_columns = ["UnitNumber", "Cycle", "RUL"]):
    
    number_engines = df["UnitNumber"].unique()
    X, y = [], []

    for engine in number_engines:

        ## get all data with engine same engine type
        temp = df[df["UnitNumber"] == engine]
        assert temp["UnitNumber"].unique() == engine

        ## loop over group
        for i in range(len(temp) - window_size + 1):

            X_temp = temp.iloc[i : (i + window_size)].drop(columns = drop_columns)
            Y_temp = temp.iloc[(i + window_size - 1)]["RUL"]
            assert len(X_temp) == 30
            X.append(X_temp.to_numpy())
            y.append(Y_temp)
            if i == (len(temp) - window_size):
                assert Y_temp == 1
    X = np.array(X)
    y = np.array(y)
    
    return X, y

In [59]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def scale_data(df):
    """
    Scales the numerical columns in the DataFrame using MinMaxScaler.

    Args:
        df (pandas.DataFrame): Input DataFrame.

    Returns:
        pandas.DataFrame: Scaled DataFrame.
    """
    scaler = MinMaxScaler()
    
    # Select float columns
    float_columns = df.select_dtypes(include=float).columns.tolist()
    
    # Scale the data
    scaled_data = scaler.fit_transform(df[float_columns])
    
    # Update the DataFrame with scaled data
    df[float_columns] = scaled_data

    return df

def create_sliding_window(df, window_size=30, drop_columns=["UnitNumber", "Cycle", "RUL"]):
    """
    Creates a sliding window of data for time series prediction.

    Args:
        df (pandas.DataFrame): Input DataFrame containing time series data.
        window_size (int): Size of the sliding window.
        drop_columns (list): List of columns to drop from the input DataFrame.

    Returns:
        tuple: A tuple containing X (input) and y (output) arrays.
    """
    number_engines = df["UnitNumber"].unique()
    X, y = [], []

    for engine in number_engines:
        # Get data for the current engine
        temp = df[df["UnitNumber"] == engine]
        assert temp["UnitNumber"].unique() == engine

        for i in range(len(temp) - window_size + 1):
            # Extract windowed data and RUL for each window
            X_temp = temp.iloc[i : (i + window_size)].drop(columns=drop_columns)
            Y_temp = temp.iloc[(i + window_size - 1)]["RUL"]
            assert len(X_temp) == window_size
            X.append(X_temp.to_numpy())
            y.append(Y_temp)
            if i == (len(temp) - window_size):
                assert Y_temp == 1

    X = np.array(X)
    y = np.array(y)

    return X, y


In [60]:
train_data = scale_data(train_data)
X_train, y_train = create_sliding_window(train_data)

In [11]:
# [TEMPLATE] - save processed data (as pickle)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_pickle(f"{config['paths']['processed_data_dir']}ex2_topic_{timestamp}.pkl")

In [12]:
# [TEMPLATE] - save data predictions (as csv)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_csv(f"{config['paths']['prediction_dir']}ex2_topic_{timestamp}.csv", sep=',', decimal='.')

In [13]:
# [TEMPLATE] - save plot results (as png)
fig = plt.figure(figsize=(9, 6))
timestamp = time.strftime("%Y%m%d-%H%M%S")
fig.savefig(f"{config['paths']['plot_dir']}ex2_topic_{timestamp}.png")

<Figure size 900x600 with 0 Axes>